# Analytical solution of Inverse Darcy Flow

we seek to compute f given the observed pressure $p_{obs}(r,t)$ for
$$
\frac{\partial p}{\partial t} - K \Delta p = f \quad \text{ on } \Omega
$$

with 

$$
p_{obs}(r,t) = (r - R_{ventricle})\cdot A \cdot sin(2 \pi f t)
$$

So:
$$
\frac{\partial p}{\partial t} = 2 \pi f (r- R_{ventricle}) \cdot A \cdot cos(2 \pi f t)
$$

and
with the laplace operator in cylindrical coordinates
$$
\Delta f(r, \theta) = \frac{1}{r}\frac{\partial}{\partial r} \left( r \frac{\partial f}{\partial r} \right) + \frac{1}{r^2} \frac{\partial^2 f}{\partial \theta^2} 
$$

$$
\Delta p(r) = \frac{1}{r}\frac{\partial }{\partial r} \left( r A \cdot sin(2 \pi f t) \right) \\
            = \frac{A}{r}sin(2 \pi f t)
$$

Finally:

$$
f = - K \Delta p + \frac{\partial p}{\partial t} \\
  = \frac{ -K A}{r} sin(2 \pi f t) + 2 A \pi f (r - R_{ventricle}) \cdot cos(2 pi f t)
$$

Let's see, if this is consistent with the numerical solution!

In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
import moola
from mshr import *
from DarcySolver import solve_darcy_on_doughnut_brain

T = 1.0           # final time
num_steps = 100    # number of time steps
dt = T/ num_steps

N = 20 # resolution
brain_radius = 0.1  # brain circle Radius
ventricle_radius = brain_radius/3

kappa = 1*1e-6    # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity

A = 1e3 / (brain_radius - ventricle_radius)
f = 1
mmHg2Pa = 133.32

brain = Circle(Point(0,0), brain_radius)
ventricle = Circle(Point(0,0), ventricle_radius)
brain = brain - ventricle
mesh = Mesh(generate_mesh(brain, N))

p_obs = Expression("A*(sqrt(x[0]*x[0] + x[1]*x[1]) - R_vent)*sin(2*pi*f*t)",
                   A=A,f=f,t=0,R_vent=ventricle_radius,degree=2)

probe_points = [
                Point(ventricle_radius + 0.25*(brain_radius - ventricle_radius), 0), 
                Point(ventricle_radius + 0.75*(brain_radius - ventricle_radius), 0), 
                ]

x_coords = np.linspace(ventricle_radius, brain_radius, 1000)
slice_points = [Point(x, 0.0) for x in x_coords]
control_space = FunctionSpace(mesh, "CG", 1)

In [ ]:
ctrls = [Function(control_space, name="control") for i in range(num_steps)]

control = [Control(c) for c in ctrls]

results = solve_darcy_on_doughnut_brain(mesh, ctrls, T, num_steps, K=K,
                                     probe_points=probe_points,
                                     slice_points=slice_points,
                                     dirichlet_boundary_skull=p_obs,
                                     dirichlet_boundary_ventricle=p_obs,
                                     p_obs=p_obs,
                                     )

rf = ReducedFunctional(results["J"], control)
problem = MoolaOptimizationProblem(rf)
f_moola = moola.DolfinPrimalVectorSet([moola.DolfinPrimalVector(c, inner_product="L2") for c in ctrls])
solver = moola.NewtonCG(problem, f_moola, options={'gtol': 1e-9,
                                                   'maxiter': 20,
                                                   'display': 3,
                                                   'ncg_hesstol': 0})

solver = moola.BFGS(problem, f_moola, options={'jtol': 1e-21,
                                               'gtol': 1e-18,
                                               'Hinit': "default",
                                               'maxiter': 20,
                                               'mem_lim': 10})
sol = solver.solve()

In [ ]:
opt_ctrls = sol['control'].data

f_ana = Expression("- K*A/(sqrt(x[0]*x[0] + x[1]*x[1]))*sin(2*pi*f*t)" +
                   "+ 2*A*pi*f*(sqrt(x[0]*x[0] + x[1]*x[1]) - R_vent)*cos(2*pi*f*t)",
                    K=K, A=A, f=f, t=0, R_vent=ventricle_radius, degree=2)

f_max = np.max([f_opt.vector().max() for f_opt in opt_ctrls])
f_min = np.min([f_opt.vector().min() for f_opt in opt_ctrls])

for i in [0,2,4,6,8, 10, 12]:
    f_max = opt_ctrls[i].vector()[:].max()
    f_min = opt_ctrls[i].vector()[:].min()

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
    plt.subplot(1,2,1)
    c = plot(opt_ctrls[i],vmin=f_min, vmax=f_max)
    plt.colorbar(c)
    plt.subplot(1,2,2)
    f_ana.t = (i +1)*dt
    c = plot(f_ana, mesh=mesh, vmin=f_min, vmax=f_max)
    plt.colorbar(c)
    plt.title(f"t = {(i +1)*dt}")


In [ ]:
results_init = solve_darcy_on_doughnut_brain(mesh, Constant(0.0), T, num_steps, K=K,
                                             probe_points=probe_points,
                                             slice_points=slice_points,
                                             dirichlet_boundary_skull=p_obs,
                                             dirichlet_boundary_ventricle=p_obs,
                                             p_obs=p_obs,
                                             )
results_ana = solve_darcy_on_doughnut_brain(mesh, f_ana, T, num_steps, K=K,
                                             probe_points=probe_points,
                                             slice_points=slice_points,
                                             dirichlet_boundary_skull=p_obs,
                                             dirichlet_boundary_ventricle=p_obs,
                                             p_obs=p_obs,
                                             )

results_opt = solve_darcy_on_doughnut_brain(mesh, opt_ctrls, T, num_steps, K=K,
                                             probe_points=probe_points,
                                             slice_points=slice_points,
                                             dirichlet_boundary_skull=p_obs,
                                             dirichlet_boundary_ventricle=p_obs,
                                             p_obs=p_obs,
                                             )
                                             

In [ ]:
p_init = results_init["slice"]
p_ana = results_ana["slice"]
p_opt = results_opt["slice"]
p_target = results_init["target_slice"]


f_ana = results_ana["f_slice"]
f_opt = results_opt["f_slice"]

times = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
idx = []

for i in range(num_steps):
    if np.isclose((i +1)*dt, times).any():
        idx.append(i)
        
idx = range(num_steps)

for i in idx:
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
    plt.subplot(1,2,1)
    plt.title("t = {}".format((i+1)*dt))
    plt.plot(x_coords, p_init[i,:]/mmHg2Pa, 
             label="p_init")
    plt.plot(x_coords, p_ana[i,:]/mmHg2Pa, 
             "--", lw=5, label="p_ana")
    plt.plot(x_coords, p_opt[i,:]/mmHg2Pa, 
             ls="--", lw=5,
             label="p_opt")
    plt.plot(x_coords, p_target[i,:]/mmHg2Pa, 
             ":", lw=5, label="p_target")
    plt.legend(loc="upper left")
    plt.xlabel("x in m")
    plt.ylabel("p in mmHg")
    plt.grid()
    plt.subplot(1,2,2)

    #ax.twinx()
    plt.plot(x_coords, 
             f_ana[i,:], 
             "*", c="red",
             label="analytical force")
    plt.plot(x_coords, 
             f_opt[i,:], 
             ls="dotted", c="green",
            label="opt. force")
    
    plt.ylabel("f in N")
    plt.grid()
    plt.legend(loc="upper center")
    